In [117]:
import os
import pandas as pd
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Получаем путь к scrapped_videos_csv
current_directory = os.getcwd()

directory = current_directory + '\Scrapped_videos_csv'

## EDA

Итак, приступим к осмотру нашего датасета. В первую очередь поймем, какие столбцы нам точно не понадобятся в дальнейшем.  
Можно убрать video_id, video_title, channel_id, channel_name, channel_description, description, как незначительные признаки.

In [136]:
df_cleaned = pd.read_csv(directory + '\\' + 'all_videos_data_no_duplicates.csv')
df_cleaned = df_cleaned.drop(columns=['video_id', 'video_title', 'channel_id', 'channel_name', 'channel_description', 'description'])

Далее посмотрим на пропуски в наших данных

In [137]:
df_cleaned.isna().sum()

published_at         0
views                0
likes             2139
comments          2035
category             5
subscribers        231
videos_count       231
source               0
comments_likes    6219
tags              7831
dtype: int64

У нас есть одинаковое кол-во пропусков в полях subscribers и videos_count. Посмотрим на эти строки

In [138]:
df_cleaned[(df_cleaned['subscribers'].isna()) & (df_cleaned['videos_count'].isna())]

,published_at,views,likes,comments,category,subscribers,videos_count,source,comments_likes,tags
100,2024-08-25T22:54:22,26097,2134.0,143.0,Разное,NaN,NaN,Rutube,NaN,NaN
869,2024-07-07T16:53:11,11422,963.0,45.0,Разное,NaN,NaN,Rutube,NaN,NaN
995,2024-06-30T18:57:11,12329,941.0,128.0,Разное,NaN,NaN,Rutube,NaN,NaN
1128,2024-06-23T10:00:02,44718,169.0,6.0,Обучение,NaN,NaN,Rutube,NaN,NaN
1465,2024-06-07T14:05:38,102806,660.0,392.0,ПМЭФ,NaN,NaN,Rutube,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4285,2025-01-27T16:00:59,80398,NaN,NaN,Новости и СМИ,NaN,NaN,Rutube,NaN,NaN
4287,2025-01-26T12:00:29,11702,NaN,NaN,Психология,NaN,NaN,Rutube,NaN,NaN
4288,2025-01-27T10:00:57,64801,NaN,NaN,Развлечения,NaN,NaN,Rutube,NaN,NaN
4289,2025-01-27T18:23:06,46480,NaN,NaN,Телепередачи,NaN,NaN,Rutube,NaN,NaN


In [139]:
df_cleaned[(df_cleaned['subscribers'].isna()) & (df_cleaned['videos_count'].isna())].value_counts('source')

source
Rutube    231
dtype: int64

Как мы видим, там, где нет кол-ва подписчиков, там и нет кол-ва видео на канале. Причем все такие случаи на Rutube.  
Вероятно это каналы, которые были удалены/скрыты и информация о них недоступна, хотя сами видео на площадке остались. Что-ж, такие у них правила.  
Глобально, раз статистика по видео в таких строках доступна, то можем оставить такие строки и заполнить subscribers и videos_count средним значением по Rutube

In [140]:
df_cleaned_rutube = df_cleaned[df_cleaned['source'] == 'Rutube']

df_cleaned['subscribers'] = df_cleaned['subscribers'].fillna(df_cleaned_rutube['subscribers'].mean())
df_cleaned['videos_count'] = df_cleaned['videos_count'].fillna(df_cleaned_rutube['videos_count'].mean())

In [141]:
df_cleaned.isna().sum()

published_at         0
views                0
likes             2139
comments          2035
category             5
subscribers          0
videos_count         0
source               0
comments_likes    6219
tags              7831
dtype: int64

Далее рассмотрим пропуски в лайках и комментариях. Прежде всего увидим, на каких платформах такие пропуски более часты

In [142]:
df_cleaned[df_cleaned['likes'].isna()].value_counts('source')

source
Rutube     2035
Youtube     104
dtype: int64

In [143]:
df_cleaned[df_cleaned['comments'].isna()].value_counts('source')

source
Rutube    2035
dtype: int64

In [144]:
df_cleaned.value_counts('source')

source
Rutube     6219
Youtube    3141
dtype: int64

In [145]:
df_cleaned[df_cleaned['source'] != 'Youtube']['category'].unique()

array(['Авто-мото', 'Новости и СМИ', 'Развлечения', 'Культура', 'Фильмы',
       'Разное', 'Обучение', 'Телепередачи', 'Политика', 'Путешествия',
       'Лайфстайл', 'Интервью', 'Животные', 'Аниме', 'Детям', 'Спорт',
       'Сериалы', 'Юмор', 'Недвижимость', 'Психология', 'Еда',
       'Видеоигры', 'Технологии и интернет',
       'Бизнес и предпринимательство', 'Музыка', 'Наука',
       'Обзоры и распаковки товаров', 'Эзотерика', 'Мультфильмы',
       'Летник RUTUBE', 'Строительство и ремонт', 'Красота', 'Хобби',
       'Техника и оборудование', 'Лайфхаки', 'Здоровье', 'Природа',
       'Гид RUTUBE: города России', 'Сад и огород', 'Спецпроекты', 'ПМЭФ',
       'Дизайн', '80 лет Великой Победе', 'Активность: «Контент-прорыв»',
       'Аудиокниги', 'Акция «Весна на экране»',
       'Акция «RUTUBE — это по любви»', 'Религия',
       'Акция «2024 — встречай!»', 'Включи Новый год'], dtype=object)

In [146]:
df_cleaned[df_cleaned['source'] == 'Youtube']['category'].unique()

array(['Comedy', 'People & Blogs', 'Film & Animation', 'Sports',
       'Science & Technology', 'News & Politics', 'Entertainment',
       'Gaming', 'Music', 'Autos & Vehicles', 'Howto & Style',
       'Education', 'Travel & Events', 'Pets & Animals', nan],
      dtype=object)

Также, мы посмотрим на общую информацию по датасету и соответствие типов данных правильным:

In [147]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9360 entries, 0 to 9359
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   published_at    9360 non-null   object 
 1   views           9360 non-null   int64  
 2   likes           7221 non-null   float64
 3   comments        7325 non-null   float64
 4   category        9355 non-null   object 
 5   subscribers     9360 non-null   float64
 6   videos_count    9360 non-null   float64
 7   source          9360 non-null   object 
 8   comments_likes  3141 non-null   object 
 9   tags            1529 non-null   object 
dtypes: float64(4), int64(1), object(5)
memory usage: 731.4+ KB


Видно, что published_at поле у нас не определилось как дата, мы исправим это в коде ниже:

In [148]:
df_cleaned['published_at'] = pd.to_datetime(df_cleaned['published_at'], errors='coerce')

И на всякий случай проверим, что у нас нет отрицательных значений:

In [153]:
numeric_cols = ['views', 'likes', 'comments', 'subscribers', 'videos_count']

for col in numeric_cols:
    if col in df_cleaned.columns:
        negative_rows = df_cleaned[df_cleaned[col] < 0]
        count = negative_rows.shape[0]
        print(f"В столбце '{col}' найдено {count} отрицательных значений.")

В столбце 'views' найдено 0 отрицательных значений.
В столбце 'likes' найдено 0 отрицательных значений.
В столбце 'comments' найдено 0 отрицательных значений.
В столбце 'subscribers' найдено 0 отрицательных значений.
В столбце 'videos_count' найдено 0 отрицательных значений.


## Теперь можем приступить к не менее интересной части EDA для обнаружения поверхностных инсайтов в данны

1) Для начала, построим корреляционную матрицу **ЧЕРЕЗ ПЛОТЛИ!!** для числовых показателей (views, likes, comments, subscribers, videos_count), чтобы понять, какие метрики связаны между собой и какие могут быть полезны для построения моделей.

In [154]:
import plotly.express as px

numeric_cols = ['views', 'likes', 'comments', 'subscribers', 'videos_count']
corr_matrix = df_cleaned[numeric_cols].corr()

fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    title="Correlation Matrix for Numeric Metrics",
    width=800,   # Ширина в пикселях
    height=600   # Высота в пикселях
)
fig.show()